In [2]:
import os
import numpy as np
import torch
from torch.nn.functional import interpolate
from tqdm import tqdm
from skimage.segmentation import find_boundaries
import matplotlib.pyplot as plt
import multiprocessing as mp

In [14]:
class Processor():
    def __init__(self, root_path, w0, sigma, inc, resize=None, start=None):
        self.root_path = root_path
        self.masks_path = os.path.join(self.root_path, "masks")

        self.masks_wt_path = os.path.join(self.root_path, "masks_wt")
        if not os.path.exists(self.masks_wt_path):
            os.mkdir(self.masks_wt_path)
        

        self.w0, self.sigma = w0, sigma
        self.inc = inc
        self.resize = resize
        self.start = start

    def process(self,):
        pbar = tqdm(os.listdir(self.masks_path))
        if self.start is not None:
            print("Starting at {}".format(self.start))
            pbar = tqdm(list(os.listdir(self.masks_path))[self.start:])

        for mask_path in pbar:
            #  new addition
            mask = np.load(os.path.join(self.masks_path, mask_path))
            mask = (mask > 0).astype(np.int32)

            if self.resize is not None:
                mask = self.resize_mask(mask).squeeze().astype(np.int32)

            mask_weight = np.expand_dims(mask, axis=0)
            endpoint = mask.shape[0]
            max_step = int(np.ceil(endpoint / self.inc))

            for i in range(max_step):
                si, ei = i*self.inc, min(endpoint, i*self.inc+self.inc)

                for j in range(max_step):
                    sj, ej = j*self.inc, min(endpoint, j*self.inc+self.inc)

                    if len(np.unique(mask[si:ei, sj:ej])) > 1:
                        mask_weight[:, si:ei, sj:ej] = self.make_weight_map(mask_weight[:, si:ei, sj:ej])
                    else:
                        mask_weight[:, si:ei, sj:ej] = 0

            plt.imshow(mask_weight.astype(np.uint8).squeeze())
            plt.show()
#                 np.save(
#                     os.path.join(self.masks_wt_path, mask_path.replace("_mask.npy", "_mask_wt.npy")),
#                     mask_weight.astype(np.uint8).squeeze()
#                 )

    def resize_mask(self, mask):
        return interpolate(torch.tensor(mask).unsqueeze(0).unsqueeze(0).float(), size=self.resize, mode="nearest").detach().numpy()

    def make_weight_map(self, masks):
        """
        Generate the weight maps as specified in the UNet paper
        for a set of binary masks.
        
        Parameters
        ----------
        masks: array-like
            A 3D array of shape (n_masks, image_height, image_width),
            where each slice of the matrix along the 0th axis represents one binary mask.

        Returns
        -------
        array-like
            A 2D array of shape (image_height, image_width)
        
        """
        nrows, ncols = masks.shape[1:]
        masks = (masks > 0).astype(int)
        distMap = np.zeros((nrows * ncols, masks.shape[0]))
        X1, Y1 = np.meshgrid(np.arange(nrows), np.arange(ncols))
        X1, Y1 = np.c_[X1.ravel(), Y1.ravel()].T
        for i, mask in enumerate(masks):
            # find the boundary of each mask,
            # compute the distance of each pixel from this boundary
            bounds = find_boundaries(mask, mode='inner')
            X2, Y2 = np.nonzero(bounds)
            xSum = (X2.reshape(-1, 1) - X1.reshape(1, -1)) ** 2
            ySum = (Y2.reshape(-1, 1) - Y1.reshape(1, -1)) ** 2
            distMap[:, i] = np.sqrt(xSum + ySum).min(axis=0)
        ix = np.arange(distMap.shape[0])
        if distMap.shape[1] == 1:
            d1 = distMap.ravel()
            border_loss_map = self.w0 * np.exp((-1 * (d1) ** 2) / (2 * (self.sigma ** 2)))
        else:
            if distMap.shape[1] == 2:
                d1_ix, d2_ix = np.argpartition(distMap, 1, axis=1)[:, :2].T
            else:
                d1_ix, d2_ix = np.argpartition(distMap, 2, axis=1)[:, :2].T
            d1 = distMap[ix, d1_ix]
            d2 = distMap[ix, d2_ix]
            border_loss_map = self.w0 * np.exp((-1 * (d1 + d2) ** 2) / (2 * (self.sigma ** 2)))
        xBLoss = np.zeros((nrows, ncols))
        xBLoss[X1, Y1] = border_loss_map
        # class weight map
        loss = np.zeros((nrows, ncols))
        w_1 = 1 - masks.sum() / loss.size
        w_0 = 1 - w_1
        loss[masks.sum(0) == 1] = w_1
        loss[masks.sum(0) == 0] = w_0
        ZZ = xBLoss + loss
        return ZZ

In [2]:
class Processor():
    def __init__(self, root_path, w0, sigma, inc, resize=None, start=None):
        self.root_path = root_path
        self.masks_path = os.path.join(self.root_path, "masks")

        self.masks_wt_path = os.path.join(self.root_path, "masks_wt")
        if not os.path.exists(self.masks_wt_path):
            os.mkdir(self.masks_wt_path)
        

        self.w0, self.sigma = w0, sigma
        self.inc = inc
        self.resize = resize
        self.start = start
    
    def driver(self, mask_path):
        mask = np.load(os.path.join(self.masks_path, mask_path))
        mask = (mask > 0).astype(np.int32)

        if self.resize is not None:
            mask = self.resize_mask(mask).squeeze().astype(np.int32)

        mask_weight = np.expand_dims(mask, axis=0)
        endpoint = mask.shape[0]
        max_step = int(np.ceil(endpoint / self.inc))

        for i in range(max_step):
            si, ei = i*self.inc, min(endpoint, i*self.inc+self.inc)

            for j in range(max_step):
                sj, ej = j*self.inc, min(endpoint, j*self.inc+self.inc)

                if len(np.unique(mask[si:ei, sj:ej])) > 1:
                    mask_weight[:, si:ei, sj:ej] = self.make_weight_map(mask_weight[:, si:ei, sj:ej])
                else:
                    mask_weight[:, si:ei, sj:ej] = 0
#         print('done')
        print(mask_weight)
        plt.imshow(mask_weight.astype(np.uint8).squeeze())
        plt.show()

    def process(self,):
        pbar = os.listdir(self.masks_path)

        if self.start is not None:
            print("Starting at {}".format(self.start))
            pbar = tqdm(list(os.listdir(self.masks_path))[self.start:])
        nprocs = mp.cpu_count()
        p = mp.Pool(processes=nprocs)
        p.map(self.driver, pbar)
#                 np.save(
#                     os.path.join(self.masks_wt_path, mask_path.replace("_mask.npy", "_mask_wt.npy")),
#                     mask_weight.astype(np.uint8).squeeze()
#                 )

    def resize_mask(self, mask):
        return interpolate(torch.tensor(mask).unsqueeze(0).unsqueeze(0).float(), size=self.resize, mode="nearest").detach().numpy()

    def make_weight_map(self, masks):
        """
        Generate the weight maps as specified in the UNet paper
        for a set of binary masks.
        
        Parameters
        ----------
        masks: array-like
            A 3D array of shape (n_masks, image_height, image_width),
            where each slice of the matrix along the 0th axis represents one binary mask.

        Returns
        -------
        array-like
            A 2D array of shape (image_height, image_width)
        
        """
        nrows, ncols = masks.shape[1:]
        masks = (masks > 0).astype(int)
        distMap = np.zeros((nrows * ncols, masks.shape[0]))
        X1, Y1 = np.meshgrid(np.arange(nrows), np.arange(ncols))
        X1, Y1 = np.c_[X1.ravel(), Y1.ravel()].T
        for i, mask in enumerate(masks):
            # find the boundary of each mask,
            # compute the distance of each pixel from this boundary
            bounds = find_boundaries(mask, mode='inner')
            X2, Y2 = np.nonzero(bounds)
            xSum = (X2.reshape(-1, 1) - X1.reshape(1, -1)) ** 2
            ySum = (Y2.reshape(-1, 1) - Y1.reshape(1, -1)) ** 2
            distMap[:, i] = np.sqrt(xSum + ySum).min(axis=0)
        ix = np.arange(distMap.shape[0])
        if distMap.shape[1] == 1:
            d1 = distMap.ravel()
            border_loss_map = self.w0 * np.exp((-1 * (d1) ** 2) / (2 * (self.sigma ** 2)))
        else:
            if distMap.shape[1] == 2:
                d1_ix, d2_ix = np.argpartition(distMap, 1, axis=1)[:, :2].T
            else:
                d1_ix, d2_ix = np.argpartition(distMap, 2, axis=1)[:, :2].T
            d1 = distMap[ix, d1_ix]
            d2 = distMap[ix, d2_ix]
            border_loss_map = self.w0 * np.exp((-1 * (d1 + d2) ** 2) / (2 * (self.sigma ** 2)))
        xBLoss = np.zeros((nrows, ncols))
        xBLoss[X1, Y1] = border_loss_map
        # class weight map
        loss = np.zeros((nrows, ncols))
        w_1 = 1 - masks.sum() / loss.size
        w_0 = 1 - w_1
        loss[masks.sum(0) == 1] = w_1
        loss[masks.sum(0) == 0] = w_0
        ZZ = xBLoss + loss
        return ZZ

In [13]:
import time
start = time.time()
processor = Processor(f"/oak/stanford/groups/deho/building_compliance/los_angeles_naip/2016_rgb_footprint_512/temp/train", 10, 4, 150)
processor.process()
end = time.time()
print(f'Time Elapsed: {end-start}')

done
done
done
done
done
done
done
done
Time Elapsed: 13.618146896362305


In [14]:
def make_weight_map(masks):
    """
    Generate the weight maps as specified in the UNet paper
    for a set of binary masks.

    Parameters
    ----------
    masks: array-like
        A 3D array of shape (n_masks, image_height, image_width),
        where each slice of the matrix along the 0th axis represents one binary mask.

    Returns
    -------
    array-like
        A 2D array of shape (image_height, image_width)

    """
    nrows, ncols = masks.shape[1:]
    masks = (masks > 0).astype(int)
    distMap = np.zeros((nrows * ncols, masks.shape[0]))
    X1, Y1 = np.meshgrid(np.arange(nrows), np.arange(ncols))
    X1, Y1 = np.c_[X1.ravel(), Y1.ravel()].T
    for i, mask in enumerate(masks):
        print(i, mask)
        # find the boundary of each mask,
        # compute the distance of each pixel from this boundary
        bounds = find_boundaries(mask, mode='inner')
        print(bounds)
        X2, Y2 = np.nonzero(bounds)
        print(X2, Y2)
        xSum = (X2.reshape(-1, 1) - X1.reshape(1, -1)) ** 2
        ySum = (Y2.reshape(-1, 1) - Y1.reshape(1, -1)) ** 2
        distMap[:, i] = np.sqrt(xSum + ySum).min(axis=0)
    ix = np.arange(distMap.shape[0])
    if distMap.shape[1] == 1:
        d1 = distMap.ravel()
        border_loss_map = w0 * np.exp((-1 * (d1) ** 2) / (2 * (sigma ** 2)))
    else:
        if distMap.shape[1] == 2:
            d1_ix, d2_ix = np.argpartition(distMap, 1, axis=1)[:, :2].T
        else:
            d1_ix, d2_ix = np.argpartition(distMap, 2, axis=1)[:, :2].T
        d1 = distMap[ix, d1_ix]
        d2 = distMap[ix, d2_ix]
        border_loss_map = w0 * np.exp((-1 * (d1 + d2) ** 2) / (2 * (sigma ** 2)))
    xBLoss = np.zeros((nrows, ncols))
    xBLoss[X1, Y1] = border_loss_map
    # class weight map
    loss = np.zeros((nrows, ncols))
    w_1 = 1 - masks.sum() / loss.size
    w_0 = 1 - w_1
    loss[masks.sum(0) == 1] = w_1
    loss[masks.sum(0) == 0] = w_0
    ZZ = xBLoss + loss
    return ZZ

In [15]:
w0=10
sigma=4
inc=150

# prototype each mask
mask = np.load(os.path.join(oak_fp, 'san_jose_naip_512', 'val', 'masks', 'm_3712142_se_10_060_20200525_256_mask.npy'))
mask = (mask > 0).astype(np.int32)

mask_weight = np.expand_dims(mask, axis=0)

endpoint = mask.shape[0]
max_step = int(np.ceil(endpoint / inc))

for i in range(max_step):
    si, ei = i*inc, min(endpoint, i*inc+inc)

    for j in range(max_step):
        sj, ej = j*inc, min(endpoint, j*inc+inc)

        if len(np.unique(mask[si:ei, sj:ej])) > 1:
            mask_weight[:, si:ei, sj:ej] = make_weight_map(mask_weight[:, si:ei, sj:ej])
        else:
            mask_weight[:, si:ei, sj:ej] = 0

0 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
[134 135 135 135 136 136 136 137 137 138 138 138 139 139 139 140 140 141
 141 141 142 142 142 143 143 144 144 144 144 145 145 145 145 146 146 147
 147 147 148 148 148 149 149] [66 64 65 67 62 63 67 61 68 59 60 68 57 58 69 56 70 54 55 70 52 53 71 51
 71 49 50 71 72 47 48 69 70 47 68 47 66 67 48 64 65 48 63]
0 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False

0 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
[ 51  51  51  52  52  52  52  53  54  55  56  57  57  57  57  58  58  58
  58  58  59  59  59  59  60  60  60  60  60  61  61  61  61  61  62  62
  62  62  63  63  63  63  63  64  64  64  64  64  65  65  65  66  66  66
  67  67  67  68  68  68  69  69  69  70  70  70  71  71  71  72  72  72
  73  73  73  74  74  74  75  75  75  76  76  76  77  77  77  78  78  78
  79  79  79  80  80  80  81  81  81  82  82  82  83  83  83  84  84  84
  85  85  85  86  86  86  87  87  87  88  88  88  89  89  89  90  90  90
  91  91  91  92  92  92  93  93  93  94  94  94  95  95  95  96  96  96
  97  97  97  98  98  98  99  

0 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
[ 55  55  56  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  77  78  79  80  80  81  82  83  84  84  85
  86  87 108 108 108 109 109 109 109 109 110 110 111 111 112 112 113 113
 114 115 116 117 118 118 118 118 119 119 119 119 119 119 120 120 120 120
 120 120 121 121 121 121 121 121 121 122 122 122 122 122 122 122 123 123
 123 123 123 123 123 124 124 124 124 125 125 125 126 126 126 127 127 127
 127 128 128 128 128 129 129 129 130 130 130 131 131 131 132 132 132 133
 133 133 134 134 134 135 135 135 136 136 136 136 137 137 137 137 137 138
 138 139 139 140 140 141 141 1